# Steps for Text Generation
- first preprocess data
- second make training sequances dataset by [seq] -> next char as lable
- make this pairs
- apply char embedding
- train LSTM model but it may a problem how to handle output from first seq as input for next sequance in lstm
- firstful we try with const seq_lenth -> after const number of character generate one char in this time its=15
- train model
- test model


In [ ]:
!pip install -U wikipedia
import numpy as np
import wikipedia as wiki
import spacy
import re
import tqdm
import tensorflow
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Preprocessing

In [ ]:
def process_text(document):
      nlp = spacy.load('en_core_web_sm')
      stopwords = nlp.Defaults.stop_words
      document = re.sub(r'\s+', ' ', document, flags=re.I)
      document = re.sub(r'\W', ' ', str(document))
      document = re.sub(r'_', ' ', str(document))
      document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
      document = document.lower()
      document = re.sub(r'\s+', ' ', document, flags=re.I)
      doc_spacy = nlp(document)
      lemma_txt = ["".join(token.lemma_) for token in tqdm.tqdm(doc_spacy)]
      lemma_no_stop_txt = [word for word in lemma_txt if word not in stopwords]
      lemma_no_stop_txt = [word for word in lemma_no_stop_txt if len(word) > 3]
      clean_txt = ' '.join(lemma_no_stop_txt)
      clean_sents_list = [x for x in nlp(clean_txt).sents]
      return clean_sents_list, clean_txt, lemma_no_stop_txt

# Collect  Data From wikipedia and Arxiv research papers

In [ ]:
# Download data (replace with your data acquisition method)
wiki.set_lang('en')

wiki_obj = wiki.page(wiki.search(['Large Language Models deep learning ', 'newest Large Language Models deep learning', 'attention mechanism ', 'transformers in Deep learning']))
data = wiki_obj.content
clean_sents_list, data_sents, data_txt = process_text(data)


In [ ]:
data_sents

In [ ]:
# [token_sent.text for token_sent in clean_sents_list][1]

In [ ]:
!pip install arxiv

In [ ]:
import arxiv

# Search for articles related to "Large Language Models" (LLMs)
llm_search = arxiv.Search(query=['Large Language Models deep learning '], max_results=100)
llm_results = list(llm_search.results())

llm_papers = ""

print("Articles related to Large Language Models:")
for result in llm_results:
    llm_papers += f"Title: {result.title}\n\nAbstract: {result.summary}\n\n"


transformer_search = arxiv.Search(query="Transformers in deep learning", max_results=100)
transformer_results = list(transformer_search.results())

transformer_papers = ""

print("\nArticles related to Transformers in deep learning:")
for result in transformer_results:
    transformer_papers += f"{result.title}Abstract: {result.summary}"

all_papers = llm_papers + transformer_papers
# print(all_papers)
data=process_text(all_papers)[1]+ ' '+ data_sents

In [ ]:
data

In [ ]:
def get_training_and_testing_sets(file_list):
    split = 0.7
    split_index = int(np.floor(len(file_list) * split))
    training = file_list
    testing = file_list[split_index:]
    return training, testing

In [ ]:
train_data,test_data=get_training_and_testing_sets(data)

In [ ]:
len(test_data)/len(data)

For training

In [ ]:
clean_sents_list, x_train_sents, x_train_clean_txt = process_text(train_data)

For testing

In [ ]:
clean_sents_list_testing, x_test_sents, x_test_clean_txt = process_text(test_data)

# Creat Training pairs Char_Based_GEN_model

In [ ]:
Seq_length=300
def create_dataset(text_data, seq_length=Seq_length):
  chars = sorted(list(set(text_data)))
  char_to_int = dict((c, i) for i, c in enumerate(chars))
  X_data = []
  y_data = []
  for i in range(0, len(text_data) - seq_length, 1):
    seq_in = text_data[i:i + seq_length]
    seq_out = text_data[i + seq_length]

    X_data.append([char_to_int[char] for char in seq_in])

    y_data.append(char_to_int[seq_out])

  return np.array(X_data),np.array(y_data),len(chars)


In [ ]:
X,y,char_unique_size=create_dataset(x_train_sents,seq_length=15)
y=y.reshape(len(y),1)
x_catagorical=tensorflow.keras.utils.to_categorical(X)
y_catagorical=tensorflow.keras.utils.to_categorical(y)
y.shape

In [ ]:
char_unique_size

# Char Based GEN Model

In [ ]:
model = tensorflow.keras.Sequential([
  Embedding(char_unique_size, 124),
#   Dropout(0.5),
  GRU(520, return_sequences=True),
  Bidirectional(layer=GRU(340, return_sequences=True,activation='tanh')),
  GlobalMaxPooling1D(),
  Dense(1024, activation='elu'),
  Dense(char_unique_size, activation='softmax'),
])

In [ ]:
model_char_based_RNN=tensorflow.keras.Sequential([
  Embedding(char_unique_size, 124),
#   Dropout(0.5),
  Bidirectional(layer=SimpleRNN(520, return_sequences=True,activation='tanh')),
  Bidirectional(layer=SimpleRNN(340, return_sequences=True,activation='tanh')),
  GlobalMaxPooling1D(),
  Dense(1024, activation='elu'),
  Dense(char_unique_size, activation='softmax'),
])

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_char_based_RNN.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
history=model.fit(X,y,epochs=10,batch_size=100)

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model,'./GRU.png')

In [ ]:
history_RNN=model_char_based_RNN.fit(X,y,epochs=20,batch_size=100)

In [ ]:
import numpy as np

def test_prediction_random_char_based_model_gen(model, input_text=test_data, seq_length=Seq_length, max_length=10):
    max_length_char_pred = []
    all_text = input_text
    chars = [char for char in input_text]
    random_start_index = np.random.randint(0, len(chars) - seq_length)
    input_text = all_text[random_start_index:random_start_index + seq_length]

    if len(chars) < seq_length:
        print("Input text is too short for the specified sequence length.")
        return None
  
    unique_chars = sorted(list(set(chars)))
    char_to_int = {c: i for i, c in enumerate(unique_chars)}
    int_to_char = {i: c for i, c in enumerate(unique_chars)}

    for i in range(max_length):
        if i > 0:
            input_text = input_text[1:] + max_length_char_pred[-1]

        if len(input_text) < seq_length:
            print("Input text is too short for the specified sequence length.")
            break

        input_seq = [char_to_int[char] for char in input_text]
        input_seq = np.array(input_seq).reshape(1, seq_length)
        prediction = model.predict(input_seq)
        predicted_char = int_to_char[np.argmax(prediction)]
        max_length_char_pred.append(predicted_char)

    true_chars = ''.join(max_length_char_pred)
    print(f'true char is {true_chars}')
    return max_length_char_pred


def test_prediction_random_char_based_model_gen(model, input_text=x_test_sents, seq_length=Seq_length,max_length):
    all_text = input_text
    chars= [char for char in input_text]

    if len(chars) < seq_length:
        print("Input text is too short for the specified sequence length.")
        return None

    random_start_index = np.random.randint(0, len(chars) - seq_length)
    input_text = all_text[random_start_index:random_start_index + seq_length]
    print(f'testing portion is ---> {input_text}')
    
    unique_chars = sorted(list(set(chars)))

    word_to_int = {w: i for i, w in enumerate(unique_chars)}
    int_to_word = {i: w for i, w in enumerate(unique_chars)}

    input_seq = [word_to_int[word] for word in input_text]
    input_seq = np.array(input_seq).reshape(1, seq_length)
    prediction = model.predict(input_seq)

    print(f'true char is {all_text[random_start_index + seq_length]}')
    return int_to_word[np.argmax(prediction)]

# Eval

GRU

In [ ]:
x_model_test.shape

In [ ]:
x_model_test,y_model_test,_=create_dataset(x_test_sents)

In [ ]:
# y_model_test=y_model_test.reshape(1,len(y_model_test))
# y_model_test.shape

In [ ]:
# model.evaluate(x_model_test,y_model_test)

In [ ]:
def plot_training_history(history):

    train_accuracy = history.history['accuracy']
#     val_accuracy = history.history['val_accuracy']
    train_loss = history.history['loss']
#     val_loss = history.history['val_loss']

    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(range(1, len(train_accuracy) + 1), train_accuracy, label='Training Accuracy')
#     plt.plot(range(1, len(val_accuracy) + 1), val_accuracy, label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training Accuracy ')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(range(1, len(train_loss) + 1), train_loss, label='Training Loss')
#     plt.plot(range(1, len(val_loss) + 1), val_loss, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training  Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Assuming 'history' contains your training history
# Call the function to plot the training history
plot_training_history(history)


In [ ]:
test_prediction_random_char_based_model_gen(model)

In [ ]:
plot_training_history(history_RNN)

RNN

In [ ]:
test_prediction_random_char_based_model_gen(model_char_based_RNN)

#### Try to make all possible sequances

In [ ]:
# its only try but take big running Duration
def create_dataset_all_sequences(text_data):
  chars = sorted(list(set(text_data)))
  char_to_int = dict((c, i) for i, c in enumerate(chars))
  X_data = []
  y_data = []

  # Loop through all starting positions in the text data
  for start_index in range(len(text_data)):
    # Loop through sequence lengths from 1 to a maximum length (can be adjusted)
    for seq_len in range(1, len(text_data) - start_index + 1):
      # Create input sequence
      input_seq = [char_to_int[char] for char in text_data[start_index:start_index + seq_len]]
      # Create target sequence (shifted by 1 character)
      target_seq = [char_to_int[char] for char in text_data[start_index + 1:start_index + seq_len + 1]]
      X_data.append(input_seq)
      y_data.append(target_seq)
  X_data = np.array(X_data)
  y_data = np.array(y_data)
  return X_data, y_data, char_to_int


# word Based generator training model

In [ ]:
x_train_sents.split()

In [ ]:
import numpy as np

Words_Seq_length = 15

def create_dataset_word_based(text_data=x_train_sents, seq_length=Words_Seq_length):
    words = text_data.split()
    unique_words = sorted(set(words))
    word_to_int = {w: i for i, w in enumerate(unique_words)}
    int_to_word = {i: w for i, w in enumerate(unique_words)}

    X_data_words = []
    y_data_words = []
    for i in range(len(words) - seq_length):
        seq_in = words[i:i + seq_length]
        seq_out = words[i + seq_length]
        X_data_words.append([word_to_int[word] for word in seq_in])
        y_data_words.append(word_to_int[seq_out])
#     z=[int_to_word[word]for word in X_data_words[0]]
#     print(z)
    return np.array(X_data_words), np.array(y_data_words), len(unique_words)


In [ ]:
X_wordSeq_training,Y_wordSEQ_training,UNique_words_len=create_dataset_word_based()

In [ ]:
model_word_generation = tensorflow.keras.Sequential()
model_word_generation.add(Embedding(UNique_words_len, 312))
model_word_generation.add(LSTM(512, input_shape=(X.shape[1], 312),return_sequences=True))
# model_word_generation.add(Dropout(0.2))
model_word_generation.add(LSTM(256))
model_word_generation.add(Dropout(0.2))
model_word_generation.add(Dense(UNique_words_len, activation='softmax'))
model_word_generation.compile(loss='categorical_crossentropy', optimizer='adam')


In [ ]:
RNN_model_word_generation=tensorflow.keras.Sequential()
RNN_model_word_generation.add(Embedding(UNique_words_len, 312))
RNN_model_word_generation.add(Bidirectional(layer=SimpleRNN(256, input_shape=(X.shape[1], 312),return_sequences=True)))
RNN_model_word_generation.add(Dropout(0.2))
RNN_model_word_generation.add(Bidirectional(layer=SimpleRNN(256)))
RNN_model_word_generation.add(Dropout(0.2))
RNN_model_word_generation.add(Dense(UNique_words_len, activation='softmax'))
RNN_model_word_generation.compile(loss='categorical_crossentropy', optimizer='adam')


In [ ]:
model_word_generation.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
RNN_model_word_generation.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history_LSTM=model_word_generation.fit(X_wordSeq_training,Y_wordSEQ_training,epochs=60,batch_size=50)

In [ ]:
history_RNN_word_based=RNN_model_word_generation.fit(X_wordSeq_training,Y_wordSEQ_training,epochs=20,batch_size=50)

In [ ]:
def test_prediction_random_word_based_model_gen(model, input_text=x_train_sents, seq_length=Words_Seq_length):
    all_text = input_text
    words = input_text.split()  # Can be commented out if not needed later

    if len(words) < seq_length:
        print("Input text is too short for the specified sequence length.")
        return None

    random_start_index = np.random.randint(0, len(words) - seq_length)
    input_text = words[random_start_index:random_start_index + seq_length]  # Extract sequence
    print(f'testing portion is ---> {input_text}')

    unique_words = sorted(list(set(words)))  # Use words if needed, otherwise remove this line

    word_to_int = {w: i for i, w in enumerate(unique_words)}
    int_to_word = {i: w for i, w in enumerate(unique_words)}

    input_seq = [word_to_int[word] for word in input_text]
    input_seq = np.array(input_seq).reshape(1, seq_length)
    prediction = model.predict(input_seq)

    print(f'true word is {all_text.split()[random_start_index + seq_length]}')
    return int_to_word[np.argmax(prediction)]


In [ ]:
len(test_data)

In [ ]:
import numpy as np

def test_prediction_random_word_based_model_gen(model, input_text=test_data, seq_length=Words_Seq_length, max_length=10):
    max_length_words_pred = []
    input_text = input_text.split()  # Split input_text into words
    all_text = input_text
    words = list(input_text)
    random_start_index = np.random.randint(0, len(words) - seq_length)
    input_text = words[random_start_index:random_start_index + seq_length]

    if len(words) < seq_length:
        print("Input text is too short for the specified sequence length.")
        return None
  
    unique_words = sorted(list(set(words)))
    word_to_int = {w: i for i, w in enumerate(unique_words)}
    int_to_word = {i: w for w, i in word_to_int.items()}

    for i in range(max_length):
        if i > 0:
            input_text = input_text[1:] + [max_length_words_pred[-1]]  

        if len(input_text) < seq_length:
            print("Input text is too short for the specified sequence length.")
            break

        input_seq = [word_to_int[word] for word in input_text]
        input_seq = np.array(input_seq).reshape(1, seq_length)
        prediction = model.predict(input_seq)
        predicted_index = np.argmax(prediction)
        
        if predicted_index >= len(int_to_word):
            print("Predicted index is out of range. Continuing with the next word.")
            continue
        
        predicted_word = int_to_word[predicted_index]
        max_length_words_pred.append(predicted_word)

    true_words = ' '.join(max_length_words_pred)  
    print(f'true word is {true_words}')
    return max_length_words_pred


In [ ]:
plot_model(model_word_generation,'/kaggle/working/model_word_gen.png')

## Test word based model based on RNN and LSTM


In [ ]:
test_prediction_random_word_based_model_gen(model_word_generation)

In [ ]:
test_prediction_random_word_based_model_gen(RNN_model_word_generation)


In [ ]:
plot_training_history(history_LSTM)

In [ ]:
plot_training_history(history_RNN_word_based)